In [3]:
import os 
import sys
import getopt
import pandas as pd
# import MySQLdb as mdb
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError
import re
import shelve
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [4]:
resultsdir = './results/regional_income/'

manf = []
rettrd = []

for fn in os.listdir(resultsdir):
    # if os.path.isfile(fn):
    m_manf = re.search('results.*manf.*\.shelf',fn)
    if m_manf:
        
        shelf = shelve.open(resultsdir+fn, 'r')
        args_dict = shelf['args_dict']
        shelf.close()
        
        manf.append(args_dict)
    else:
        m_rettrd = re.search('results.*rettrd.*\.shelf',fn)
        if m_rettrd:

            shelf = shelve.open(resultsdir+fn, 'r')
            args_dict = shelf['args_dict']
            shelf.close()
            
            rettrd.append(args_dict)

In [64]:
def dict_to_dfs(args_dict):
    # dictionary that will be filled, then used to fill a df
    fill_dict = {}
    
    # first add datapoint entries
    fill_dict['data'] = {}
    fill_dict['data']['x'] = []
    fill_dict['data']['y'] = []
    fill_dict['data']['case'] = []
    fill_dict['data']['geofips'] = []
    fill_dict['data']['industry'] = []
    
    # manually select & name the coordinate pair cases
    for x_name, y_name, case in [('X_train', 'Y_train', 'data_train'),
                                   ('X_full', 'Y_full', 'data_full'),
                                   ('X_test', 'Y_test', 'data_test'),
                                   ('X_train', 'ypred_train', 'pred_train'),
                                   ('X_full', 'ypred_full', 'pred_full'),
                                   ('X_test', 'ypred_test', 'pred_test'),
                                   ('Xproj_train', 'yproj_train', 'proj_train'),
                                   ('Xproj_full', 'yproj_full', 'proj_full')]:
        for x,y in zip(args_dict[x_name],args_dict[y_name]):
            fill_dict['data']['x'].append(x)
            fill_dict['data']['y'].append(y)
            fill_dict['data']['case'].append(case)
            fill_dict['data']['geofips'].append(args_dict['geofips'])
            fill_dict['data']['industry'].append(args_dict['industry'])

    # loop to fill the rest of the data
    fill_dict['attributes'] = {}
    fill_dict['fit_scores'] = {}
    for key in args_dict:
        # skip the arrays/lists
        if type(args_dict[key]) == type([]) or type(args_dict[key]) == type(np.array([])):
            continue
            
        # for dictionary entries (i.e. the fit scores)
        elif type(args_dict[key]) == type({}):
            if 'geofips' not in fill_dict['fit_scores']:
                fill_dict['fit_scores']['geofips'] = []
            if 'industry' not in fill_dict['fit_scores']:
                fill_dict['fit_scores']['industry'] = []
            if 'case' not in fill_dict['fit_scores']:
                fill_dict['fit_scores']['case'] = []
                    
            fill_dict['fit_scores']['geofips'].append(args_dict['geofips'])
            fill_dict['fit_scores']['industry'].append(args_dict['industry'])
            fill_dict['fit_scores']['case'].append(key)
            
            for subkey in args_dict[key]:
                if subkey not in fill_dict['fit_scores']:
                    fill_dict['fit_scores'][subkey] = []
                fill_dict['fit_scores'][subkey].append(args_dict[key][subkey])
        
        else:
            fill_dict['attributes'][key] = [args_dict[key]]
            
    df_dict = {}
    for entry in fill_dict:
        df_dict[entry] = pd.DataFrame(fill_dict[entry])
        
    return df_dict

In [62]:
def write_to_sql(df, name):
    try:
        # print 'Connecting to database...'
        # con = mdb.connect('localhost', 'danielj', '', 'ecotest')
        engine = create_engine(
            "mysql+mysqldb://danielj:@localhost/ecotest")
    except:
        print 'Error connecting to db:', sys.exc_info()[0]
        return 0
    try:
        # print 'Writing to database table:%s'%name
        df.to_sql(con=engine, name=name,
                  if_exists='append', flavor='mysql')
    except OperationalError as err:
        print 'Error writing to db:', sys.exc_info()[0]
        print err.message
        return 0
    return 1

In [63]:
def write_all_to_sql(tmp_dict):
    n_tmp_dict = len(tmp_dict)

    for entry,count in zip(tmp_dict,range(1,n_tmp_dict+1)):
        df_dict = dict_to_dfs(entry)
        for name in df_dict:
            write_to_sql(df_dict[name], name)

        sys.stdout.write('\rProgress: %d/%d' % (count, n_tmp_dict))
        sys.stdout.flush()

    print '\ndone!'

        

In [66]:
write_all_to_sql(rettrd)

Progress: 375/375
done!
